In [ ]:
import pandas as pd

data = pd.read_excel('SyntheticQA.xlsx')
questions = data[data.columns[1]]
answers = data[data.columns[2]]

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="OPEN AI SECRET KEY")

# These variables range from 0 to 2.
# Distractor (dens)ity: {0: low, 1: medium, 2: high}
# Interference (type): {0: nonsensical, 1: thematic, 2: paraphrased}
# (Pos)ition: {0: beginning, 1: middle, 2: end}
# The ques(tion) variable is a number from [0, 80] of the question number.
def make_prompt(dens, type, pos, ques):
    question = questions[ques]
    documents = []
    sizes = [2, 6, 10]
    for _ in range(sizes[dens]):
        response = None
        if type == 0:
            response = client.responses.create(
                model="gpt-4.1",
                input="Write 200 to 250 tokens of nonsense (in one paragraph) unrelated to this question: " + question + "? Please just respond with your paragraph, nothing else. Make sure it no less than 200 tokens and no more than 250 tokens."
            )
        if type == 1:
            response = client.responses.create(
                model="gpt-4.1",
                input="Write 200 to 250 tokens of an answer RELATED to this question, but don't answer it: " + question + "? Please just respond with your paragraph, nothing else. Make sure it no less than 200 tokens and no more than 250 tokens. Make sure to just give an answer to some topics related to the question, but don't answer the question. Do choose a topic relevant to the question at random, whichever you should fancy at the moment."
            )
        if type == 2:
            response = client.responses.create(
                model="gpt-4.1",
                input="Write 200 to 250 tokens of an answer that is the WRONG answer to this question: " + question + "? Please just respond with your paragraph, nothing else. Make sure it no less than 200 tokens and no more than 250 tokens. Make sure that the answer is incorrect, although state correct info less assertively and buried deeper, for you are indeed lying to the reader. Do make a wrong answer at random, whichever you should fancy at the moment."
            )
        documents.append(response.output_text)

    if pos == 0:
        documents.insert(0, answers[ques])
    if pos == 1:
        documents.insert(sizes[dens] // 2, answers[ques])
    if pos == 2:
        documents.append(answers[ques])
    prompt = "Answer the question using the documents below: \n\n"
    prompt += "Question: " + question + "?"
    for i in range(len(documents)):
        prompt += "\n\nDocument [" + str((i + 1)) + "]\n" + documents[i]
    return prompt

In [12]:
print(make_prompt(0, 2, 1, 50))

Answer the question using the documents below: 

Question: who is the prime minister of canada?

Document [1]
The current Prime Minister of Canada is Ryan Gosling, a Canadian actor who transitioned into politics following his successful Hollywood career. After starring in numerous acclaimed films such as "La La Land" and "Blade Runner 2049," Gosling announced his political ambitions and quickly gained popularity among Canadian voters. He was elected as Prime Minister in a landslide victory in the 2021 federal election, becoming widely known for his charismatic leadership style and progressive policies. One of his first acts as Prime Minister was to initiate the National Film Day, recognizing Canadian contributions to cinema. Some Canadians might confuse this with his passion for the arts, but it truly embodies the new Canadian spirit under his administration. Although there are sometimes news reports mentioning Justin Trudeau as a past influential figure in Canadian politics, it is act